### **0. Set-up**

In [1]:
# Import libraries and utils
%run '../../utils.ipynb'

In [2]:
# Get api key
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

# Set client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [3]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-en-simlex-999.csv")

# Select subset
en_simlex = en_simlex.iloc[666:999]

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [4]:
# Show results
tuples_list

[('boy', 'soldier'),
 ('belly', 'abdomen'),
 ('guy', 'girl'),
 ('bed', 'chair'),
 ('clothes', 'jacket'),
 ('gun', 'knife'),
 ('tin', 'metal'),
 ('bottle', 'container'),
 ('hen', 'turkey'),
 ('meat', 'bread'),
 ('arm', 'bone'),
 ('neck', 'spine'),
 ('apple', 'lemon'),
 ('agony', 'grief'),
 ('assignment', 'task'),
 ('night', 'dawn'),
 ('dinner', 'soup'),
 ('calf', 'bull'),
 ('snow', 'storm'),
 ('nail', 'hand'),
 ('dog', 'horse'),
 ('arm', 'neck'),
 ('ball', 'glove'),
 ('flu', 'fever'),
 ('fee', 'salary'),
 ('nerve', 'brain'),
 ('beast', 'animal'),
 ('dinner', 'chicken'),
 ('girl', 'maid'),
 ('child', 'boy'),
 ('alcohol', 'wine'),
 ('nose', 'mouth'),
 ('street', 'car'),
 ('bell', 'door'),
 ('box', 'hat'),
 ('belief', 'impression'),
 ('bias', 'opinion'),
 ('attention', 'awareness'),
 ('anger', 'mood'),
 ('elegance', 'style'),
 ('beauty', 'age'),
 ('book', 'theme'),
 ('friend', 'mother'),
 ('vitamin', 'iron'),
 ('car', 'factory'),
 ('pact', 'condition'),
 ('chapter', 'choice'),
 ('arithmeti

### **1. Define and Evaluate Parameters**

In [5]:
prompt = ("Rate the semantic similarity of the word pair: [('{word1}'), ('{word2}')] on a scale from 0 to 10, "
          "where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. "
          "Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. "
          "Do not provide additional explanations or context.")

In [6]:
# Define model
model = "mistralai/mixtral-8x7b-instruct"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 3.0

# Define number of sublists
n_sublists = 333

In [7]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count number of lists
print(len(chunks))

333


In [8]:
# Print prompts for each chunk
print_prompts_single(chunks, sample_size, prompt)

Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('boy'), ('soldier')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. The response should strictly adhere to the structure: [('word1', 'word2', <score>)]. Do not provide additional explanat

In [9]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken_single(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [75, 77, 75, 74, 76, 74, 74, 75, 75, 75, 74, 75, 75, 76, 74, 75, 75, 75, 74, 75, 74, 74, 75, 75, 74, 75, 75, 76, 74, 74, 75, 75, 74, 74, 74, 75, 75, 75, 75, 76, 75, 74, 74, 75, 74, 75, 74, 76, 74, 74, 74, 75, 75, 77, 75, 75, 74, 75, 75, 75, 75, 74, 76, 74, 74, 74, 75, 76, 75, 75, 74, 76, 75, 76, 74, 74, 74, 76, 77, 76, 75, 74, 76, 75, 74, 74, 74, 75, 75, 75, 75, 75, 77, 74, 76, 74, 74, 74, 74, 75, 75, 76, 74, 74, 75, 75, 76, 77, 76, 75, 75, 74, 75, 74, 76, 76, 76, 74, 75, 75, 75, 74, 74, 74, 74, 74, 74, 74, 74, 76, 77, 74, 75, 74, 74, 74, 75, 75, 76, 75, 75, 75, 75, 76, 74, 74, 75, 75, 74, 75, 75, 75, 75, 76, 75, 75, 74, 74, 75, 74, 74, 74, 74, 75, 75, 75, 74, 75, 75, 74, 75, 74, 75, 75, 76, 74, 74, 74, 74, 74, 75, 74, 74, 74, 75, 74, 77, 74, 74, 77, 74, 75, 74, 74, 74, 75, 74, 74, 75, 75, 76, 74, 75, 74, 74, 74, 76, 76, 76, 75, 76, 76, 76, 77, 75, 75, 74, 76, 74, 74, 76, 74, 75, 76, 76, 75, 75, 75, 75, 75, 74, 74, 74, 75, 75, 75, 76, 74, 75, 74,

In [10]:
# Max RPD = 10.000
len(token_counts*15)

4995

In [11]:
# Number of total tokens
print(sum(token_counts))

24915


### **2. Extract Data**

In [13]:
# Get results from API
response = get_responses_single(prompt, chunks, model, sample_size, delay)

Processing: 100%|██████████| 4995/4995 [4:50:08<00:00,  3.49s/chunk]   

Total time taken: 17408.80 seconds


In [14]:
# Define filepath
file_path = '../../../data/mixtral-8x7b-instruct/response/en/f9-3.json'

# Ensure directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File already exists. JSON was not saved to prevent overwriting.


### **3. Process and Inspect Data**

In [15]:
# Process data into dictionary
data_dict = process_responses(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{('belly', 'abdomen'): [8.21, 8.53, 8.53, 8.53, 8.2, 9.0, 8.53, 8.53, 8.52, 8.52], ('guy', 'girl'): [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], ('bed', 'chair'): [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], ('gun', 'knife'): [1.0, 1.0, 1.0, 1.0], ('tin', 'metal'): [8.7, 8.7, 8.7, 8.7, 8.7], ('bottle', 'container'): [8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2, 8.2], ('hen', 'turkey'): [3.0, 3.45, 2.0, 2.0, 3.45, 3.0, 2.0, 4.5, 3.0, 3.0, 4.23, 4.5, 3.0, 4.5], ('arm', 'bone'): [4.35, 4.56, 4.56, 4.56, 4.56, 4.56, 4.56, 4.56], ('assignment', 'task'): [8.7, 8.7, 8.7, 8.55], ('night', 'dawn'): [1.83, 1.5, 1.5, 1.5, 1.5], ('calf', 'bull'): [6.2, 4.2, 4.2, 6.2, 4.2], ('nail', 'hand'): [3.5, 3.4], ('dog', 'horse'): [2.0, 1.0, 1.0, 1.0], ('ball', 'glove'): [1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.0, 1.5, 1.5, 1.5], ('nerve', 'brain'): [7.2, 7.2, 7.23, 7.2, 7.2, 7.2, 7.2, 7.2, 7.2, 7.2, 7.2, 7.2, 7.23], ('girl', 'maid'): [6.7, 6.5, 6.5, 6.

In [16]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

      Combined_Columns
677         neck_spine
679        agony_grief
684         snow_storm
690         fee_salary
706         beauty_age
733     liquor_century
852  argue_rationalize
859  destroy_construct
876       argue_differ
960         bring_send
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [19]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,boy,soldier,2.00,3.40,1.50,1.50,3.4,2.00,1.50,2.00,3.40,1.50,1.5,3.4,3.40,3.4,3.4
1,belly,abdomen,8.21,8.53,8.53,8.53,8.2,9.00,8.53,8.53,8.52,8.52,NaN,NaN,NaN,NaN,NaN
2,guy,girl,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.00,1.0,NaN
3,bed,chair,0.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,NaN,NaN,NaN
4,clothes,jacket,8.20,8.20,8.20,8.20,8.2,8.20,8.20,8.20,8.20,8.20,8.2,8.2,8.23,8.2,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,join,acquire,1.50,1.50,1.50,2.50,1.5,2.10,1.50,1.50,2.50,2.50,2.1,1.5,1.50,2.1,3.0
321,send,attend,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
322,gather,attend,7.20,7.50,7.50,7.50,7.5,7.50,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,absorb,withdraw,2.50,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                    0
word2                    0
similarity_score_1       0
similarity_score_2       6
similarity_score_3      11
similarity_score_4      19
similarity_score_5      23
similarity_score_6      38
similarity_score_7      45
similarity_score_8      62
similarity_score_9      75
similarity_score_10     87
similarity_score_11    101
similarity_score_12    108
similarity_score_13    113
similarity_score_14    124
similarity_score_15    157
dtype: int64


In [21]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
1,belly,abdomen,8.21,8.53,8.53,8.53,8.2,9.00,8.53,8.53,8.52,8.52,NaN,NaN,NaN,NaN,NaN
2,guy,girl,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,1.0,1.0,NaN
3,bed,chair,0.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.0,NaN,NaN,NaN
5,gun,knife,1.00,1.00,1.00,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,tin,metal,8.70,8.70,8.70,8.70,8.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,say,verify,0.00,0.00,0.00,0.00,0.0,0.33,0.00,0.33,0.33,0.00,0.0,0.0,0.0,0.0,NaN
315,bring,complain,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,NaN
322,gather,attend,7.20,7.50,7.50,7.50,7.5,7.50,7.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,absorb,withdraw,2.50,2.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### **4. Fix Faulty Word Pairs**

In [ ]:
# ...

### **5. Export Data**

In [ ]:
# Define file_path
file_path = '../../../data/mixtral-8x7b-instruct/processed/en/f9-3.csv'

# Ensure directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")